In [6]:
# 라이브러리 import
import os
import pandas as pd
import numpy as np
import math
# 셀레늄
from selenium.webdriver.common.alert import Alert
from selenium import webdriver  # 라이브러리(모듈) 가져오라
# pip install chromedriver_autoinstaller
from selenium.webdriver.common.keys import Keys
from selenium.webdriver import ActionChains as AC
from selenium.webdriver.chrome.options import Options
import subprocess
import shutil
import chromedriver_autoinstaller
# tqdm : for문 진행상황 체크
from tqdm import tqdm, tqdm_notebook
from tqdm.notebook import tqdm
# 정규표현식(regular expression) : 문자(알파벳,한글), 숫자, 특수기호 정제 및 추출
import re
import random
from time import sleep
import time
# 워닝 무시
import warnings
warnings.filterwarnings('ignore')

In [ ]:
# # 크롬 옵션
# options = webdriver.ChromeOptions()
# # 크롬 윈도우 사이즈 조절
# options.add_argument("--window-size=1600,1200") # window-size -> 기본 : 1920,1080
# # 기상청을 키자
# chrome_path = chromedriver_autoinstaller.install()
# driver = webdriver.Chrome(chrome_path, options=options)
# time.sleep(2)
# driver.get("https://www.weather.go.kr/w/index.do")
# time.sleep(4)

# # 관측 기후 클릭
# driver.find_element_by_css_selector('#wnuri-menu > ul > li.d1-item08.accordionsecond-wrap > a').click()
# time.sleep(5)
# # 육상 클릭
# driver.find_element_by_css_selector('#wnuri-menu > ul > li.d1-item08.accordionsecond-wrap > ul > li.d2-item01.accordionsecond-wrap > a').click()
# time.sleep(4)
# # 과거관측 클릭
# driver.find_element_by_css_selector('#wnuri-menu > ul > li.d1-item08.accordionsecond-wrap > ul > li.d2-item01.accordionsecond-wrap.acco-on > ul > li.d3-item03 > a').click()
# time.sleep(3)

# # 기상 기록 수집하기
# for u in [1, 3, 5, 7, 9]:
#     for v in range(1, 8):
#         x = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span'.format(u, v))[0].text
#         if x != ' ':
#             a = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(1)'.format(u+1, v))[0].text
#             b = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(4)'.format(u+1, v))[0].text
#             c = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(7)'.format(u+1, v))[0].text
#             d = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(10)'.format(u+1, v))[0].text
#             e = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(13)'.format(u+1, v))[0].text
#             평균기온.append(float(a[5:].replace('℃', '')))
#             최고기온.append(float(b[5:].replace('℃', '')))
#             최저기온.append(float(c[5:].replace('℃', '')))
#             평균운량.append(d[5:])
#             일강수량.append(e[5:])

경주=283,영천=281,구미=279,대구=143,포항=138,통영=162,진주=192,울산=152,부산=159,목포=165,여수=168,완도=170,진도군=268

영광=252,순천=174,해남=261,고흥=262,안동=136,남해=295,함양군=264,거제=294,밀양=288,창원=155,보령=235,금산=238,전주=146

군산=140,부안=243,임실=244,순창=254,고창=251,광주=156,울진=130,영주=272,문경=273,영덕=277,의성=278,대관령=100,청주=131

충주=127,제천=221,보은=226,홍성=177,대전=133,서산=129,천안=232,서울=108,동두천=98,파주=99,인천=112,수원=119,강화=201

양평=202,이천=203,춘천=101,철원=95,원주=114,영월=121,인제=211,홍천=212,강릉=105,속초=90,정선군=217

In [15]:
N = int(input('지역에 해당하는 정수 stn을 입력하시오 :'))
평균기온 = [];최고기온 = [];최저기온 = [];평균운량 = [];일강수량 = [];날짜 = []
# 크롬 옵션
options = webdriver.ChromeOptions()
# 크롬 윈도우 사이즈 조절
options.add_argument("--window-size=1600,1200") # window-size -> 기본 : 1920,1080
# 기상청을 키자
chrome_path = chromedriver_autoinstaller.install()
driver = webdriver.Chrome(chrome_path, options=options)
time.sleep(1)
for year in range(2016, 2021):
    if year != 2020:
        for month in range(1, 13):
            driver.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn={}&yy={}&mm={}&ob'.format(N, year, month))
            time.sleep(4)
            for u in [1, 3, 5, 7, 9, 11]:
                for v in range(1, 8):
                    try:
                        x = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span'.format(u, v))[0].text
                        if x != ' ':
                            a = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(1)'.format(u+1, v))[0].text
                            b = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(4)'.format(u+1, v))[0].text
                            c = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(7)'.format(u+1, v))[0].text
                            d = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(10)'.format(u+1, v))[0].text
                            e = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(13)'.format(u+1, v))[0].text
                            평균기온.append(float(a[5:].replace('℃', '')))
                            최고기온.append(float(b[5:].replace('℃', '')))
                            최저기온.append(float(c[5:].replace('℃', '')))
                            평균운량.append(d[5:])
                            일강수량.append(e[5:])
                            날짜.append(str(year) + str(month) + x)
                    except:
                        pass
            time.sleep(random.randint(6, 11))
    else:
        for month in range(1, 10):
            driver.get('https://www.weather.go.kr/w/obs-climate/land/past-obs/obs-by-day.do?stn={}&yy={}&mm={}&ob'.format(N, year, month))
            time.sleep(4)
            for u in [1, 3, 5, 7, 9, 11]:
                for v in range(1, 8):
                    try:
                        x = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span'.format(u, v))[0].text
                        if x != ' ':
                            a = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(1)'.format(u+1, v))[0].text
                            b = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(4)'.format(u+1, v))[0].text
                            c = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(7)'.format(u+1, v))[0].text
                            d = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(10)'.format(u+1, v))[0].text
                            e = driver.find_elements_by_css_selector('body > div.container > section > div > div.cont-wrap > div.cmp-past-obs > div:nth-child(3) > div > table > tbody > tr:nth-child({}) > td:nth-child({}) > span:nth-child(13)'.format(u+1, v))[0].text
                            평균기온.append(float(a[5:].replace('℃', '')))
                            최고기온.append(float(b[5:].replace('℃', '')))
                            최저기온.append(float(c[5:].replace('℃', '')))
                            평균운량.append(d[5:])
                            일강수량.append(e[5:])
                            날짜.append(str(year) + str(month) + x)
                    except:
                        pass
            time.sleep(random.randint(6, 11))

지역에 해당하는 정수 stn을 입력하시오 :112


In [16]:
# 데이터프레임 만들기
테스트 = pd.DataFrame({'날짜':날짜,
                    '평균기온':평균기온,
                    '최고기온':최고기온,
                    '최저기온':최저기온,
                    '평균운량':평균운량,
                    '일강수량':일강수량})
테스트

,날짜,평균기온,최고기온,최저기온,평균운량,일강수량
0,201611일,3.1,5.7,-0.4,7.4,-
1,201612일,7.4,11.1,4.0,6.0,-
2,201613일,5.9,7.9,4.8,9.4,-
3,201614일,2.2,5.2,-1.8,5.3,-
4,201615일,-1.9,0.2,-3.6,4.0,-
...,...,...,...,...,...,...
1730,2020926일,19.3,23.9,15.7,3.9,-
1731,2020927일,19.9,24.4,15.3,0.8,-
1732,2020928일,20.2,24.1,16.4,1.8,-
1733,2020929일,19.6,22.7,16.8,6.9,-


In [19]:
# 2020년 9월 29일이랑 30일 빼기
날씨데이터 = 테스트[:-2]
날씨데이터

,날짜,평균기온,최고기온,최저기온,평균운량,일강수량
0,201611일,3.1,5.7,-0.4,7.4,-
1,201612일,7.4,11.1,4.0,6.0,-
2,201613일,5.9,7.9,4.8,9.4,-
3,201614일,2.2,5.2,-1.8,5.3,-
4,201615일,-1.9,0.2,-3.6,4.0,-
...,...,...,...,...,...,...
1728,2020924일,20.9,24.6,17.3,7.4,-
1729,2020925일,19.9,24.4,16.4,4.5,0.0mm
1730,2020926일,19.3,23.9,15.7,3.9,-
1731,2020927일,19.9,24.4,15.3,0.8,-


In [20]:
날씨데이터.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1733 entries, 0 to 1732
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   날짜      1733 non-null   object 
 1   평균기온    1733 non-null   float64
 2   최고기온    1733 non-null   float64
 3   최저기온    1733 non-null   float64
 4   평균운량    1733 non-null   object 
 5   일강수량    1733 non-null   object 
dtypes: float64(3), object(3)
memory usage: 81.4+ KB


In [21]:
날씨데이터['일강수량'][0] # 작대기가 어떻게 생겼는가

' -'

In [22]:
날씨데이터['평균운량'][0]

'7.4'

In [29]:
z = 날씨데이터['일강수량'][1729]
z

'0.0mm'

In [32]:
z[:-2]

'0.0'

In [26]:
# 평균운량에서 str로 된 실수값을 float화, ' -'이런 놈은 0으로 치환
A = []
for i in 날씨데이터['평균운량']:
    if i == ' -':
        A.append(0)
    else:
        A.append(float(i))

In [33]:
# 일강수량에서 ' -'이런 놈은 0으로 치환, str에 mm붙은 건 적절한 슬라이싱
B = []
for j in 날씨데이터['일강수량']:
    if j == ' -':
        B.append(0)
    else:
        B.append(float(j[:-2]))

In [34]:
날씨데이터['평균운량'] = A
날씨데이터['일강수량'] = B
날씨데이터

,날짜,평균기온,최고기온,최저기온,평균운량,일강수량
0,201611일,3.1,5.7,-0.4,7.4,0.0
1,201612일,7.4,11.1,4.0,6.0,0.0
2,201613일,5.9,7.9,4.8,9.4,0.0
3,201614일,2.2,5.2,-1.8,5.3,0.0
4,201615일,-1.9,0.2,-3.6,4.0,0.0
...,...,...,...,...,...,...
1728,2020924일,20.9,24.6,17.3,7.4,0.0
1729,2020925일,19.9,24.4,16.4,4.5,0.0
1730,2020926일,19.3,23.9,15.7,3.9,0.0
1731,2020927일,19.9,24.4,15.3,0.8,0.0


In [35]:
날씨데이터.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1733 entries, 0 to 1732
Data columns (total 6 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   날짜      1733 non-null   object 
 1   평균기온    1733 non-null   float64
 2   최고기온    1733 non-null   float64
 3   최저기온    1733 non-null   float64
 4   평균운량    1733 non-null   float64
 5   일강수량    1733 non-null   float64
dtypes: float64(5), object(1)
memory usage: 81.4+ KB
